In [1]:
'Vehicle Detection using Yolov8 Model'
import gradio as gr
from gradio.components import Video
import cv2
import numpy as np
from collections import defaultdict
from ultralytics import YOLO
import supervision as sv

In [2]:
def vehicles_detection(video_path):
    """
    Detect and classify vehicles in a video using YOLOv8 model.

    Parameters
    ----------
    video_path : str
        Path to the input video file.

    Returns
    -------
    None
    """
    # Load the YOLOv8 model
    model = YOLO('yolov8n.pt')
    cap = cv2.VideoCapture(video_path)
    
    # Define the line coordinates
    start = sv.Point(182, 254)
    end = sv.Point(462, 254)
    
    # Store the track history
    track_history = defaultdict(list)
    # Create a dictionary to keep track of objects that have crossed the line
    crossed_objects = {}
    
    # Open a video sink for the output video
    video_info = sv.VideoInfo.from_video_path(video_path)
    with sv.VideoSink("output_video.mp4", video_info) as sink:
        while cap.isOpened():
            success, frame = cap.read()
            if not success:
                break
            
            # Run YOLOv8 tracking on the frame, persisting tracks between frames
            results = model.track(
                frame,
                classes=[2, 3, 5, 7],
                persist=True,
                save=True,
                tracker="bytetrack.yaml"
            )

            # Get the boxes and track IDs
            boxes = results[0].boxes.xywh.cpu()
            track_ids = results[0].boxes.id.int().cpu().tolist()

            # Visualize the results on the frame
            annotated_frame = results[0].plot()
            detections = sv.Detections.from_yolov8(results[0])

            # Plot the tracks and count objects crossing the line
            for box, track_id in zip(boxes, track_ids):
                x, y, w, h = box
                track = track_history[track_id]
                track.append((float(x), float(y)))  # x, y center point
                if len(track) > 30:  # retain 30 tracks for 30 frames
                    track.pop(0)

                # Check if the object crosses the line
                if start.x < x < end.x and abs(y - start.y) < 5:  # Assuming objects cross horizontally
                    if track_id not in crossed_objects:
                        crossed_objects[track_id] = True

                    # Annotate the object as it crosses the line
                    cv2.rectangle(
                        annotated_frame,
                        (int(x - w / 2), int(y - h / 2)),
                        (int(x + w / 2), int(y + h / 2)),
                        (0, 255, 0),
                        2
                    )

            # Draw the line on the frame
            cv2.line(annotated_frame, (start.x, start.y), (end.x, end.y), (0, 255, 0), 2)

            # Write the count of objects on each frame
            count_text = f"Vehicles Crossed: {len(crossed_objects)}"
            cv2.putText(annotated_frame, count_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            # Write the frame with annotations to the output video
            sink.write_frame(annotated_frame)

    # Release the video capture
    cap.release()

In [3]:
# Create Gradio interface
iface = gr.Interface(
    fn=vehicles_detection,
    inputs=Video(),  # Use Video component from gradio.components
    outputs=Video(), # Use Video component from gradio.components
    title="Vehicle Detection and Classification",
    description="Upload a video to detect and classify vehicles using YOLOv8."
)

In [4]:
if __name__ == "__main__":
    iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.



0: 384x640 7 cars, 208.7ms
Speed: 6.0ms preprocess, 208.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 209.8ms
Speed: 5.0ms preprocess, 209.8ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 204.9ms
Speed: 4.1ms preprocess, 204.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 232.2ms
Speed: 4.1ms preprocess, 232.2ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 205.4ms
Speed: 7.0ms preprocess, 205.4ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 209.3ms
Speed: 7.0ms preprocess, 209.3ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 225.6ms
Speed:

Results saved to runs\detect\track

0: 384x640 8 cars, 206.8ms
Speed: 3.0ms preprocess, 206.8ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 8 cars, 207.4ms
Speed: 3.2ms preprocess, 207.4ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 8 cars, 200.9ms
Speed: 3.2ms preprocess, 200.9ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 9 cars, 204.3ms
Speed: 3.2ms preprocess, 204.3ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 9 cars, 209.1ms
Speed: 5.0ms preprocess, 209.1ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 9 cars, 197.7ms
Speed: 3.0ms preprocess, 197.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

Results saved to runs\detect\track

0: 384x640 5 cars, 197.9ms
Speed: 3.2ms preprocess, 197.9ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 202.2ms
Speed: 2.4ms preprocess, 202.2ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 206.8ms
Speed: 3.1ms preprocess, 206.8ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 211.3ms
Speed: 7.0ms preprocess, 211.3ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 202.8ms
Speed: 4.0ms preprocess, 202.8ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 211.2ms
Speed: 4.0ms preprocess, 211.2ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

Results saved to runs\detect\track

0: 384x640 6 cars, 215.8ms
Speed: 5.1ms preprocess, 215.8ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 209.6ms
Speed: 3.2ms preprocess, 209.6ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 203.2ms
Speed: 2.0ms preprocess, 203.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 201.8ms
Speed: 3.6ms preprocess, 201.8ms inference, 5.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 211.0ms
Speed: 4.1ms preprocess, 211.0ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 202.2ms
Speed: 2.2ms preprocess, 202.2ms inference, 4.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

...


0: 384x640 6 cars, 231.3ms
Speed: 10.0ms preprocess, 231.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 206.1ms
Speed: 5.1ms preprocess, 206.1ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 211.8ms
Speed: 3.2ms preprocess, 211.8ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 1 truck, 208.5ms
Speed: 2.4ms preprocess, 208.5ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 6 cars, 206.2ms
Speed: 3.0ms preprocess, 206.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 205.5ms
Speed: 3.1ms preprocess, 205.5ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 7 cars, 214.

Results saved to runs\detect\track

0: 384x640 4 cars, 212.6ms
Speed: 7.0ms preprocess, 212.6ms inference, 2.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 206.9ms
Speed: 2.3ms preprocess, 206.9ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 206.7ms
Speed: 4.0ms preprocess, 206.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 212.3ms
Speed: 4.3ms preprocess, 212.3ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 212.3ms
Speed: 6.0ms preprocess, 212.3ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 197.7ms
Speed: 2.2ms preprocess, 197.7ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

Results saved to runs\detect\track

0: 384x640 4 cars, 209.0ms
Speed: 4.0ms preprocess, 209.0ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 206.1ms
Speed: 6.5ms preprocess, 206.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 203.4ms
Speed: 3.6ms preprocess, 203.4ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 209.2ms
Speed: 3.0ms preprocess, 209.2ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 207.2ms
Speed: 4.2ms preprocess, 207.2ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 4 cars, 200.7ms
Speed: 4.2ms preprocess, 200.7ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

Results saved to runs\detect\track

0: 384x640 3 cars, 207.0ms
Speed: 6.4ms preprocess, 207.0ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 3 cars, 225.0ms
Speed: 3.0ms preprocess, 225.0ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 3 cars, 212.7ms
Speed: 2.0ms preprocess, 212.7ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 2 cars, 1 truck, 256.1ms
Speed: 3.2ms preprocess, 256.1ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 3 cars, 387.2ms
Speed: 4.9ms preprocess, 387.2ms inference, 2.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 3 cars, 235.9ms
Speed: 3.0ms preprocess, 235.9ms inference, 4.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\det

Speed: 3.1ms preprocess, 216.4ms inference, 3.1ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 204.0ms
Speed: 2.1ms preprocess, 204.0ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 201.4ms
Speed: 3.2ms preprocess, 201.4ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 211.2ms
Speed: 2.2ms preprocess, 211.2ms inference, 3.2ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 208.8ms
Speed: 6.6ms preprocess, 208.8ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track

0: 384x640 5 cars, 199.8ms
Speed: 3.1ms preprocess, 199.8ms inference, 3.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs\detect\track
